# Introduction
A key feature of PorePy is that algorithmic (or automatic) differentiation, AD, is used to translate  partial differential equations into a system of linear equations.
This tutorial provides a short introduction to algorithmic differentiation and an overview of the AD-module included in PorePy. 

Further description is provided in other tutorials: *pdes with ad*  explains how to use the module to solve (mixed-dimensional) PDEs. For an example where the AD module has been used to solve non-linear compressible flow, see the tutorial:  "compressible_flow_with_automatic_differentiation"


In [3]:
import numpy as np
import scipy.sparse as sps

from porepy.numerics.ad.forward_mode import Ad_array
import porepy.numerics.ad.functions as af
import porepy as pp

# AD module
Algorithmic differentiation is used to compute the derivative of a function. This is achieved by augmenting variables with a field representing the derivative in addition to the value of the variable. Evaluation of a function of such variables is performed by (repeatedly) applying the chain rule while keeping track of both values and derivatives.

## Scalar variables

We initiate a variable $x = 2$ by giving a pair (val, jac) to the Ad_array class. val is the value at which the function will be evaluated and jac =1 since $\frac{d x}{dx} = 1$.

In [14]:
x = Ad_array(2, 1)

We can now define a function $y=x^2 + 3$ 

In [15]:
y = x**2 + 3

To obtain the function value and the derivative we can call .val and .jac

In [16]:
print('y value is: ', y.val)
print('dy/dx is: ', y.jac)

y value is:  7.0
dy/dx is:  4.0


$y$ is also an AD variable as a function of $x$. We can use it to declare further functions, e.g., $h(x) = e^{y(x)}$. To take the exponential of an Ad_array we need to call the exponential function found in the AD module

In [17]:
h = af.exp(y)
print('h value is: ', h.val)
print('dh/dx is: ', h.jac)

h value is:  1096.6331584284585
dh/dx is:  4386.532633713834


If we know the value and Jacobian of $y$ we could alternatively skip initiating $x$ and initiate $y$ directly:

In [18]:
y = Ad_array(7, 4)
h = af.exp(y)
print('h value is: ', h.val)
print('dh/dx is: ', h.jac)

h value is:  1096.6331584284585
dh/dx is:  4386.532633713834


## Vector variables
The Ad_array class also support arrays.

In [169]:
x = Ad_array(np.array([3,2,1]), sps.diags([1,1,1]))

As for the scalar case, it is straightforward to define functions using normal Python programming. Let us declare the function
$$y(x) = Ax + x^2 - b$$
which has the Jacobian
$$ J_y(x) = A + 2 \text{diag}(x)$$
With this notation we mean $x^2 = [x_1^2, x_2^2, x_3^2]$, and $\text{diag}(x)$ is a matrix with $x$ on the diagonal and zeros elsewhere.

In [170]:
A = sps.csc_matrix(np.array([[3,2,1],
                             [2,6,1],
                             [2,3,4]]))
b =  Ad_array(np.array([1,0,0]), sps.diags([0,0,0]))
y = A*x  + x**2 - b

print('Analytic y value: ')
print(np.array([22, 23, 17]))
print('Analytic y Jacobian:')
print(np.array([[9,2,1],[2,10,1],[2,3,6]]),'\n')
print('Ad y value: ')
print(y.val)
print('Ad y Jacobian:')
print(y.jac.A)


Analytic y value: 
[22 23 17]
Analytic y Jacobian:
[[ 9  2  1]
 [ 2 10  1]
 [ 2  3  6]] 

Ad y value: 
[22. 23. 17.]
Ad y Jacobian:
[[ 9.  2.  1.]
 [ 2. 10.  1.]
 [ 2.  3.  6.]]


## Equations
An equation is defined by setting
$$y(x) = 0.$$
We can solve it using the Newton-Rhapson scheme, which, on residual form, reads 
$$J_y(x^{k}) (x^{k+1} - x^{k}) = - y(x^{k}).$$

In [171]:
tol = 1e-5
residuals = list()
for i in range(5): 
    x = x - sps.linalg.spsolve(y.jac, y.val)
    y = A*x  + x**2 - b
    residuals.append(np.linalg.norm(y.val))

print(f"After five iterations, the solution is {x.val}. The residuals were ")
print(residuals)


After five iterations, the solution is [ 0.39609763 -0.11506208 -0.11506208]. The residuals were 
[5.121415047969018, 0.37208984847741916, 0.010886193063849024, 1.2643761102354224e-05, 1.7809260833326555e-11]


Conceptually, this example illustrates how mixed-dimensional PDEs are solved in PorePy:
1. Define ad variables and parameters/discretization matrices
2. Combine them into equations
3. Solve the nonlinear problem using Newton-Rhapson
In the mixed-dimensional case, some additional book-keeping is required. Specificially, variables etc. are defined on subdomains and interfaces of the mixed-dimensional grid as shown in the following.